In [26]:
from pymfe.mfe import MFE
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import os
import openml
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import time
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
from sklearn.datasets import load_iris

In [111]:
class dataset_OpenML(object):
    """
    Create an OpenML specific version of the dataset class.
    """

    def __init__(self, data_id):
        # read dataset
        self.X, self.y, self.name = self.__get_data(data_id)

    def __get_data(self, data_id):
        # Retrieve dataset from OpenMl
        dataset = openml.datasets.get_dataset(data_id)
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute
        )
        X = self.impute_missing(X, categorical_indicator)

        numerical_target = is_numeric_dtype(y)

        # if numerical_target:
        #     # Scale labels to range from [0,1]
        #     labels = self.scale_labels(y)
        #
        # else:
        #     # Transform labels into categorical encoding
        #     labels = self.encode_labels(y)

        return X, y, dataset.name

    def impute_missing(self, X, categorical_indicator):
        object_cols = {col: 'object' for col in np.array(X.columns[categorical_indicator])}
        X = X.astype(object_cols)
        dtypes = dict(X.dtypes)
        numeric_features = list(X.columns[~np.array(categorical_indicator)])
        numeric_transformer = SimpleImputer(strategy="mean")

        categorical_features = list(X.columns[categorical_indicator])
        categorical_transformer = SimpleImputer(strategy="constant", fill_value="missing")
        type_specific_preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, numeric_features),
                ("cat", categorical_transformer, categorical_features),
            ]
        )
        prepr_np = type_specific_preprocessor.fit_transform(X)
        prepr_df = pd.DataFrame(prepr_np, columns=numeric_features + categorical_features)
        prepr_df = prepr_df.astype(dtypes)
        return prepr_df

    def get_arrays(self):
        return np.asarray(self.X), np.asarray(self.y)

In [27]:
dataset = openml.datasets.get_dataset(40996)
X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format="dataframe", target=dataset.default_target_attribute)

False

In [22]:
openml_df = openml.datasets.list_datasets(output_format="dataframe")
openml_df[openml_df['name'] == 'Fashion-MNIST']


,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
40996,40996,Fashion-MNIST,1,2506,active,ARFF,7000.0,10.0,7000.0,10.0,785.0,70000.0,0.0,0.0,784.0,1.0


In [102]:
dataset = openml.datasets.get_dataset(6332)
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="dataframe", target=dataset.default_target_attribute)

In [110]:
object_cols = {col:'object' for col in np.array(X.columns[categorical_indicator])}
X.astype(object_cols)
     


customer                object
job_number               int64
grain_screened          object
ink_color               object
proof_on_ctd_ink        object
blade_mfg               object
cylinder_division       object
paper_type              object
ink_type                object
direct_steam            object
solvent_type            object
type_on_cylinder        object
press_type              object
press                   object
unit_number              uint8
cylinder_size           object
paper_mill_location     object
plating_tank            object
proof_cut              float64
viscosity              float64
caliper                 object
ink_temperature        float64
humifity               float64
roughness              float64
blade_pressure         float64
varnish_pct            float64
press_speed            float64
ink_pct                float64
solvent_pct            float64
ESA_Voltage            float64
ESA_Amperage           float64
wax                    float64
hardener

In [91]:
type(np.asarray(dat.X)[0][18])

str

In [82]:
list(y.values)
X.values
type(np.asarray(y.values))

numpy.ndarray

In [101]:
dat.X.dtypes

job_number                int64
unit_number               uint8
proof_cut               float64
viscosity               float64
ink_temperature         float64
humifity                float64
roughness               float64
blade_pressure          float64
varnish_pct             float64
press_speed             float64
ink_pct                 float64
solvent_pct             float64
ESA_Voltage             float64
ESA_Amperage            float64
wax                     float64
hardener                float64
roller_durometer        float64
anode_space_ratio       float64
customer               category
grain_screened         category
ink_color              category
proof_on_ctd_ink       category
blade_mfg              category
cylinder_division      category
paper_type             category
ink_type               category
direct_steam           category
solvent_type           category
type_on_cylinder       category
press_type             category
press                  category
cylinder

In [113]:
dat = dataset_OpenML(6332)
X, y = dat.get_arrays()
dat.X.isna().any()

job_number             False
unit_number            False
proof_cut              False
viscosity              False
ink_temperature        False
humifity               False
roughness              False
blade_pressure         False
varnish_pct            False
press_speed            False
ink_pct                False
solvent_pct            False
ESA_Voltage            False
ESA_Amperage           False
wax                    False
hardener               False
roller_durometer       False
anode_space_ratio      False
customer               False
grain_screened         False
ink_color              False
proof_on_ctd_ink       False
blade_mfg              False
cylinder_division      False
paper_type             False
ink_type               False
direct_steam           False
solvent_type           False
type_on_cylinder       False
press_type             False
press                  False
cylinder_size          False
paper_mill_location    False
plating_tank           False
caliper       

In [ ]:
MFE()

In [127]:
ft[1]

[0.0644665970503495,
 0.10681140835811544,
 1.8906490168129426,
 1.1292266412928182,
 0.06851851851851852,
 0.6388888888888888,
 0.14190819669842775,
 0.7834513256923659,
 nan,
 1.0555555555555556,
 0.05311932802798223,
 0.1138621702147878,
 0.9824740868386409,
 0.040506088515330614,
 0.06671764453615259,
 8.041616174393983,
 336.617403777315,
 489113.3109367248,
 6100467.726470403,
 0.437037037037037,
 0.04876152508180617,
 16.811619141368737,
 0.5,
 0.10999438818457402,
 236.98775706880298,
 2923.455265578917,
 985.2232856727205,
 232.81300286147047,
 2871.671854114693,
 14.594594594594595,
 22.046715944012288,
 237.50329471793214,
 2.8146829213837705,
 1.1201025958781865,
 135.14753809038834,
 167.02936594662572,
 80,
 9.425,
 2.919922835810667,
 0.0125,
 0.024691846377417833,
 20440.027572427574,
 34891.8882469992,
 0.5,
 0.053033008588991036,
 1.5893049981681275,
 0.6240740740740741,
 0.08510910898876127,
 16.316076910268155,
 176.80384252769804,
 587.796858974359,
 7065.420386307

In [130]:
ft[1]

[0.0644665970503495,
 0.10681140835811544,
 1.8906490168129426,
 1.1292266412928182,
 0.06851851851851852,
 0.6388888888888888,
 0.14190819669842775,
 0.7834513256923659,
 nan,
 1.0555555555555556,
 0.05311932802798223,
 0.1138621702147878,
 0.9824740868386409,
 0.040506088515330614,
 0.06671764453615259,
 8.041616174393983,
 336.617403777315,
 489113.3109367248,
 6100467.726470403,
 0.437037037037037,
 0.04876152508180617,
 16.811619141368737,
 0.5,
 0.10999438818457402,
 236.98775706880298,
 2923.455265578917,
 985.2232856727205,
 232.81300286147047,
 2871.671854114693,
 14.594594594594595,
 22.046715944012288,
 237.50329471793214,
 2.8146829213837705,
 1.1201025958781865,
 135.14753809038834,
 167.02936594662572,
 80,
 9.425,
 2.919922835810667,
 0.0125,
 0.024691846377417833,
 20440.027572427574,
 34891.8882469992,
 0.5,
 0.053033008588991036,
 1.5893049981681275,
 0.6240740740740741,
 0.08510910898876127,
 16.316076910268155,
 176.80384252769804,
 587.796858974359,
 7065.420386307

In [131]:
pd.DataFrame(data=[ft[1]], index=[dat.name], columns=ft[0])

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
cylinder-bands,0.064467,0.106811,1.890649,1.129227,0.068519,0.638889,0.141908,0.783451,NaN,1.055556,...,0.09947,0.041933,0.074902,489113.310937,6.100466e+06,0.00641,0.018849,0.386204,0.577778,0.007808


In [118]:
dat = dataset_OpenML(6332)
X, y = dat.get_arrays()
mfe = MFE()
mfe.fit(X, y)
ft = mfe.extract(cat_cols='auto', suppress_warnings=True)
ft

C:\Users\20172613\Anaconda3\envs\cap_sel\lib\site-packages\pymfe\_internal.py:1566: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")


(['attr_conc.mean',
  'attr_conc.sd',
  'attr_ent.mean',
  'attr_ent.sd',
  'attr_to_inst',
  'best_node.mean',
  'best_node.sd',
  'can_cor.mean',
  'can_cor.sd',
  'cat_to_num',
  'class_conc.mean',
  'class_conc.sd',
  'class_ent',
  'cor.mean',
  'cor.sd',
  'cov.mean',
  'cov.sd',
  'eigenvalues.mean',
  'eigenvalues.sd',
  'elite_nn.mean',
  'elite_nn.sd',
  'eq_num_attr',
  'freq_class.mean',
  'freq_class.sd',
  'g_mean.mean',
  'g_mean.sd',
  'gravity',
  'h_mean.mean',
  'h_mean.sd',
  'inst_to_attr',
  'iq_range.mean',
  'iq_range.sd',
  'joint_ent.mean',
  'joint_ent.sd',
  'kurtosis.mean',
  'kurtosis.sd',
  'leaves',
  'leaves_branch.mean',
  'leaves_branch.sd',
  'leaves_corrob.mean',
  'leaves_corrob.sd',
  'leaves_homo.mean',
  'leaves_homo.sd',
  'leaves_per_class.mean',
  'leaves_per_class.sd',
  'lh_trace',
  'linear_discr.mean',
  'linear_discr.sd',
  'mad.mean',
  'mad.sd',
  'max.mean',
  'max.sd',
  'mean.mean',
  'mean.sd',
  'median.mean',
  'median.sd',
  'mi

In [23]:
dataset = openml.datasets.get_dataset(40996)
X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format="dataframe", target=dataset.default_target_attribute)
